# <center> Lab 02: Linear Regression 실습 <center/>


$$ H(x) = Wx + b $$

$$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $$

 - $H(x)$: 주어진 $x$ 값에 대해 예측을 어떻게 할 것인가
 - $cost(W, b)$: $H(x)$ 가 $y$ 를 얼마나 잘 예측했는가

## 모듈을 사용하지 않은 단일 선형 회귀 코드

In [2]:
# import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# pytorch의 random seed를 고정할 때 쓰는 함수
torch.manual_seed(1)

# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])
# 모델 초기화
W = torch.zeros(1, requires_grad=True)  # requires_grad=True는 epochs마다 메모리에 연산했던 것들을 track하는 것.
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)  # SGD로 optimizer 정의, learning rate는 0.01 

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train * W + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    # 항상 붙어다니는 3줄
    optimizer.zero_grad()  # gradient 초기화
    cost.backward()  # 기울기 계산
    optimizer.step() # 경사하강법으로 최적화

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch    0/1000 W: 0.093, b: 0.040 Cost: 4.666667
Epoch  100/1000 W: 0.873, b: 0.289 Cost: 0.012043
Epoch  200/1000 W: 0.900, b: 0.227 Cost: 0.007442
Epoch  300/1000 W: 0.921, b: 0.179 Cost: 0.004598
Epoch  400/1000 W: 0.938, b: 0.140 Cost: 0.002842
Epoch  500/1000 W: 0.951, b: 0.110 Cost: 0.001756
Epoch  600/1000 W: 0.962, b: 0.087 Cost: 0.001085
Epoch  700/1000 W: 0.970, b: 0.068 Cost: 0.000670
Epoch  800/1000 W: 0.976, b: 0.054 Cost: 0.000414
Epoch  900/1000 W: 0.981, b: 0.042 Cost: 0.000256
Epoch 1000/1000 W: 0.985, b: 0.033 Cost: 0.000158


+ 초기화한 w와 b, 목적 함수, 손실 함수 전부 정의해야됨.   
+ torch.optim()을 써서 최적화를 진행함

---------------------------------------------------------------------------------------------------------------------------------------

## `nn.Module`을 사용한 단일 선형 회귀 코드

### 먼저 모델 클래스 만들어주기 

In [3]:
# 모델 클래스 만들기
class LinearRegressionModel(nn.Module):  # nn.model 클래스를 상속받음
    def __init__(self):
        super().__init__()                # 상속받은 nn.model의 함수를 이용
        self.linear = nn.Linear(1, 1)     # 단순 선형 회귀이므로 input=1, ouput=1

    def forward(self, x):                 # 모델이 학습 데이터를 받아서 forward시키는 함수
        return self.linear(x)

In [4]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim

# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

    
# 모델 초기화
model = LinearRegressionModel()

    
# hypothesis, cost 함수 정의
hypothesis = model(x_train)
cost = F.mse_loss(hypothesis, y_train) # 이번에는 간단하게 내장 함수를 이용함.(직접 수식 구현 x)
print(cost)


# optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01) # 최적화 기법으로 SGD 채택    
    
    
    
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    # cost로 H(x) 개선
    optimizer.zero_grad() # gradient 초기화
    cost.backward()       # 기울기 계산
    optimizer.step()      # 경사하강법으로 최적화 
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

tensor(2.1471, grad_fn=<MseLossBackward>)
Epoch    0/1000 W: 0.578, b: -0.413 Cost: 2.147149
Epoch  100/1000 W: 1.066, b: -0.150 Cost: 0.003239
Epoch  200/1000 W: 1.052, b: -0.118 Cost: 0.002002
Epoch  300/1000 W: 1.041, b: -0.093 Cost: 0.001237
Epoch  400/1000 W: 1.032, b: -0.073 Cost: 0.000764
Epoch  500/1000 W: 1.025, b: -0.057 Cost: 0.000472
Epoch  600/1000 W: 1.020, b: -0.045 Cost: 0.000292
Epoch  700/1000 W: 1.016, b: -0.035 Cost: 0.000180
Epoch  800/1000 W: 1.012, b: -0.028 Cost: 0.000111
Epoch  900/1000 W: 1.010, b: -0.022 Cost: 0.000069
Epoch 1000/1000 W: 1.008, b: -0.017 Cost: 0.000043


#### cost가 점점 줄어듦